# SHAPE

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=3)

from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')

In [ ]:
import simushape  as ss
import numpy as np
# the plan is to write my predictions in a react file
if False:
    data = ss.get_all_data('data/RNA16.react','data/RNA16.dbn')
    res=[]
    for e in data.keys(): 
        train = data.keys()
        train.remove(e)
        model = ss.make_model(data,train,False,3,3)
        graph = ss.eden_rna.sequence_dotbracket_to_graph(data[e][1],data[e][2])
        my_react = np.array(ss.predict(model,graph))
        res.append(">%s"%e)
        res.append('\n'.join(["%s\t%.4f" % (i,e) for i,e in enumerate(my_react)]))
        res.append('')

    with open("data/shaker_prediction.react",'w') as f: f.write('\n'.join(res))


# DRAW FUNCTION FOR THE SHAPE DATA

In [ ]:
from graphlearn01.utils import draw
import numpy as np
from scipy.stats import pearsonr as pear
from scipy.stats import spearmanr as spear
import simushape  as ss


def annotate(g,shap):
    n=g.nodes()
    n.sort()
    for e,i in zip(n,shap):
        g.node[e]["importance"]= [i]
        g.node[e]["none"]= ''
    return g

def draw3(graph, shape_list):
    graphs=[annotate(graph.copy(),shape) for shape in shape_list  ]
    draw.graphlearn(graphs,size=15,layout="RNA",vertex_label='none',edge_alpha=0.05,vertex_size=150,vertex_border=False, scoretricks=True)
    

def mycorr(train, prediction):
    train = np.array(train)
    prediction = np.array(prediction)
    predict2, train2 = ss.mask(prediction,train)
    p =  pear(train2,predict2)[0]
    s =  spear(train2,predict2)[0]
    return "pearson: %.3f spearman %.3f variances: %.3f %.3f" % (p,s, train2.var(), predict2.var())


klmer = ss.read_react("data/klmer_prediction.react")
shaker = ss.read_react("data/shaker_prediction.react")
RNA16 =  ss.read_react("data/RNA16.react")
allreacts = [RNA16, shaker,klmer]
RNA16_complete = ss.get_all_data('data/RNA16.react','data/RNA16.dbn')# load includes sanity check;; name -> react,sequence,dotbracket



for e in RNA16_complete:
    print "rna name", e
    graph = ss.eden_rna.sequence_dotbracket_to_graph(RNA16_complete[e][1],RNA16_complete[e][2])
    graph.graph["structure"] = RNA16_complete[e][2]
    draw3(graph,[ react[e] for react in allreacts ])
    print "\t\t\t\t\t\t\t",mycorr(RNA16[e],shaker[e]),"\t\t\t", mycorr(RNA16[e],klmer[e])
  
    
    
    



---